##### [2022-DataMining]
# Cross Validation
* Hold-out
* Repeated Hold-out
* K-Fold CV
* Stratified K-Fold CV
* Repeated Stratified K-Fold CV
##  --최민영--

In [ ]:
import pandas as pd
import numpy as np

## sklearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split ## Hold-out CV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import roc_auc_score

In [119]:
file_path = "C:/Users/wl498/Documents/datamine/dataset-of-10s.csv"
spotify = pd.read_csv(file_path)

input_features = ['energy', 'danceability', 'acousticness', 'loudness','instrumentalness']
target = 'target'

#### 모델 정의

In [3]:
knn_params = {'n_neighbors': 15, 'metric': 'minkowski', 'p':3}
knn = KNeighborsClassifier(**knn_params)

### **Hold-out**
#### 1) train/test 분할


In [4]:
X,y = spotify[input_features], spotify[target]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 999) ## test size

#### 2) train/test feature scaling

In [5]:
scaler = RobustScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.fit_transform(X_test)

### 3) 모델 적합 및 예측
X_train에 대하여 학습시키고 X_test에 대하여 예측 수행

In [6]:
knn.fit(X_train_sc, y_train)

KNeighborsClassifier(n_neighbors=15, p=3)

In [7]:
knn.fit(X_train_sc, y_train)
y_pred = knn.predict(X_test_sc)

In [10]:
from evaluation import BCMetrics
bc_eval = BCMetrics(y_true = y_test, y_pred = y_pred, pos_label=1)
bc_eval.evaluate(return_df=True)

acc       tpr       tnr        f1       ppv       npv      bacc  \
0  0.809896  0.914255  0.711246  0.823757  0.749561  0.897698  0.812751   

        fpr       fnr      plr       nlr  
0  0.085745  0.288754  10.6625  2.463158

In [11]:
def eval_func(y_true, y_pred, pos_label = 1):
    bc_eval = BCMetrics(y_true = y_true, y_pred = y_pred, pos_label= pos_label)
    auc_score = roc_auc_score(y_true, y_pred)
    bc_eval_result = bc_eval.evaluate(True)
    bc_eval_result.insert( 0, "auc", auc_score)
    return bc_eval_result

In [12]:
eval_func(y_test,y_pred)

auc       acc       tpr       tnr        f1       ppv       npv  \
0  0.812751  0.809896  0.914255  0.711246  0.823757  0.749561  0.897698   

       bacc       fpr       fnr      plr       nlr  
0  0.812751  0.085745  0.288754  10.6625  2.463158

In [13]:
def binary_clf_eval(y_true, y_prob, thres = None, pos_label=1):
    labels = np.unique(y_true)
    neg_label = np.delete(labels, pos_label).item()
    
    if y_prob.ndim >1:
        y_prob = y_prob[:,1]
    t = 0.5 if thres == None else thres
    y_pred = np.where( y_prob >= 0.5, 1, 0)
    bc_eval = BCMetrics(y_true = y_true, y_pred = y_pred)
    auc_score = roc_auc_score(y_true, y_prob)
    bc_eval_result = bc_eval.evaluate(True)
    bc_eval_result.insert( 0, "auc", auc_score)
    return bc_eval_result

In [14]:
y_prob = knn.predict_proba(X_test_sc)
binary_clf_eval(y_test,y_prob)

auc       acc       tpr       tnr        f1       ppv       npv  \
0  0.87448  0.809896  0.914255  0.711246  0.823757  0.749561  0.897698   

       bacc       fpr       fnr      plr       nlr  
0  0.812751  0.085745  0.288754  10.6625  2.463158

In [15]:
binary_clf_eval(y_test,y_pred)

auc       acc       tpr       tnr        f1       ppv       npv  \
0  0.812751  0.809896  0.914255  0.711246  0.823757  0.749561  0.897698   

       bacc       fpr       fnr      plr       nlr  
0  0.812751  0.085745  0.288754  10.6625  2.463158

In [16]:
binary_clf_eval(y_test,y_prob,thres=.6)

auc       acc       tpr       tnr        f1       ppv       npv  \
0  0.87448  0.809896  0.914255  0.711246  0.823757  0.749561  0.897698   

       bacc       fpr       fnr      plr       nlr  
0  0.812751  0.085745  0.288754  10.6625  2.463158

### **Repeated Hold-out**

In [17]:
R = 10
scaler = RobustScaler()
rHO_result = pd.DataFrame()
for r in range(R):   
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3) 
    X_train_sc = scaler.fit_transform(X_train)
    X_test_sc = scaler.transform(X_test)
        
    knn.fit(X_train_sc, y_train)
    y_prob = knn.predict(X_test_sc)
    
    bc_eval_result = binary_clf_eval(y_test,y_prob)
    bc_eval_result.insert(0, "repeat_ind", r)
    
    rHO_result = pd.concat( [rHO_result, bc_eval_result ])

In [18]:
rHO_result 

repeat_ind       auc       acc       tpr       tnr        f1       ppv  \
0           0  0.800998  0.800000  0.896842  0.705155  0.816092  0.748682   
0           1  0.807590  0.808854  0.894251  0.720930  0.825984  0.767401   
0           2  0.805920  0.803125  0.917735  0.694106  0.819656  0.740517   
0           3  0.816150  0.814063  0.907249  0.725051  0.826615  0.759144   
0           4  0.803592  0.804167  0.913990  0.693194  0.824299  0.750638   
0           5  0.822769  0.823958  0.910586  0.734952  0.839810  0.779244   
0           6  0.801822  0.802604  0.895661  0.707983  0.820634  0.757205   
0           7  0.819115  0.819271  0.893971  0.744259  0.832124  0.778281   
0           8  0.818959  0.817708  0.911299  0.726619  0.831407  0.764393   
0           9  0.823052  0.825000  0.904374  0.741729  0.841060  0.786030   

        npv      bacc       fpr       fnr        plr       nlr  
0  0.874680  0.800998  0.103158  0.294845   8.693878  2.391608  
0  0.868790  0.807590  0.105749  0.279070   8.456311  2.583333  
0  0.898684  0.805920  0.082265  0.305894  11.155844  2.269103  
0  0.891114  0.816150  0.092751  0.274949   9.781609  2.637037  
0  0.888591  0.803592  0.086010  0.306806  10.626506  2.259386  
0  0.888889  0.822769  0.089414  0.265048  10.183908  2.772908  
0  0.869677  0.801822  0.104339  0.292017   8.584158  2.424460  
0  0.874847  0.819115  0.106029  0.255741   8.431373  2.910204  
0  0.893805  0.818959  0.088701  0.273381  10.273810  2.657895  
0  0.880862  0.823052  0.095626  0.258271   9.457447  2.871901

Repeated Hold Out 결과 요약

In [19]:
rHO_result.drop('repeat_ind',1).apply([np.mean, np.std])[['auc','acc','tpr','tnr','f1']]

C:\Users\Hallym\AppData\Local\Temp\ipykernel_5832\2123964977.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  rHO_result.drop('repeat_ind',1).apply([np.mean, np.std])[['auc','acc','tpr','tnr','f1']]


auc       acc       tpr       tnr        f1
mean  0.811997  0.811875  0.904596  0.719398  0.827768
std   0.008858  0.009332  0.008873  0.018573  0.008323

### **KFold CrossValidation**
#### 1) KFold CV

In [20]:
from sklearn.model_selection import KFold
K = 5
scaler = RobustScaler()
kfold = KFold(n_splits=K, shuffle=True,)
for k, (ind1, ind2) in enumerate(kfold.split(X,y)):
    print(k)
    print(ind1.shape, ind2.shape)

0
(5118,) (1280,)
1
(5118,) (1280,)
2
(5118,) (1280,)
3
(5119,) (1279,)
4
(5119,) (1279,)


In [21]:
X if isinstance(X,np.ndarray) == True else np.array(X)

array([[ 6.2600e-01,  7.4100e-01,  2.0000e-02, -4.8260e+00,  0.0000e+00],
       [ 2.4700e-01,  4.4700e-01,  8.7100e-01, -1.4661e+01,  8.1400e-01],
       [ 4.1500e-01,  5.5000e-01,  1.6100e-01, -6.5570e+00,  0.0000e+00],
       ...,
       [ 8.0400e-01,  7.1900e-01,  1.3200e-02, -4.5810e+00,  3.0200e-06],
       [ 1.7700e-01,  6.0000e-01,  9.8900e-01, -1.6070e+01,  8.6800e-01],
       [ 1.2300e-01,  1.2100e-01,  9.6400e-01, -2.3025e+01,  6.9600e-01]])

In [22]:
kFold_result = pd.DataFrame()
for k, (train_ind, test_ind) in enumerate(kfold.split(X, y)): 
    X = X if isinstance(X,np.ndarray) == True else np.array(X)
    y = y if isinstance(X,np.ndarray) == True else np.array(y)
        
    X_train, X_test = X[train_ind], X[test_ind]
    y_train, y_test = y[train_ind], y[test_ind]
        
    X_train_sc = scaler.fit_transform(X_train)
    X_test_sc = scaler.transform(X_test) 
        
    knn.fit(X_train_sc, y_train)
    
    y_prob = knn.predict_proba(X_test_sc)
        
    bc_eval_result = binary_clf_eval(y_test,y_prob)
    bc_eval_result.insert(0, "fold_ind", k)
    kFold_result = pd.concat( [kFold_result, bc_eval_result ])

In [23]:
kFold_result

fold_ind       auc       acc       tpr       tnr        f1       ppv  \
0         0  0.859989  0.815625  0.897833  0.731861  0.830946  0.773333   
0         1  0.881371  0.813281  0.893939  0.727419  0.831572  0.777339   
0         2  0.870710  0.812500  0.914516  0.716667  0.825328  0.751989   
0         3  0.875284  0.808444  0.894309  0.728916  0.817844  0.753425   
0         4  0.870375  0.810790  0.908815  0.706924  0.831711  0.766667   

        npv      bacc       fpr       fnr        plr       nlr  
0  0.875472  0.814847  0.102167  0.268139   8.787879  2.729412  
0  0.865643  0.810679  0.106061  0.272581   8.428571  2.668639  
0  0.899240  0.815591  0.085484  0.283333  10.698113  2.529412  
0  0.881603  0.811612  0.105691  0.271084   8.461538  2.688889  
0  0.879760  0.807869  0.091185  0.293076   9.966667  2.412088

In [24]:
kFold_result.drop('fold_ind', axis =1).apply([np.mean, np.std])

auc       acc       tpr       tnr        f1       ppv       npv  \
mean  0.871546  0.812128  0.901882  0.722357  0.827480  0.764551  0.880343   
std   0.007844  0.002695  0.009281  0.010362  0.006003  0.011475  0.012235   

          bacc       fpr       fnr       plr       nlr  
mean  0.812120  0.098118  0.277643  9.268554  2.605688  
std   0.003158  0.009281  0.010362  1.014753  0.131807

### 층화 K겹 교차 검증: ```StratifiedKFold```

In [36]:
from sklearn.model_selection import StratifiedKFold
stkFold = StratifiedKFold(n_splits=5,shuffle=True)
for train_ind, test_in in stkFold.split(X,y):
    print(np.unique(y[train_ind],return_counts=True),np.unique(y[test_ind],return_counts=True))

stkFold_result = pd.DataFrame()
scaler = RobustScaler()
for k, (train_ind, test_ind) in enumerate(stkFold.split(X, y)): 
       
    bc_eval_result = eval_func(y_test,y_pred)
    bc_eval_result.insert(0, "fold_ind", k)
    stkFold_result = pd.concat( [stkFold_result, bc_eval_result ])

(array([0, 1], dtype=int64), array([2559, 2559], dtype=int64)) (array([0, 1], dtype=int64), array([1066, 1066], dtype=int64))
(array([0, 1], dtype=int64), array([2559, 2559], dtype=int64)) (array([0, 1], dtype=int64), array([1066, 1066], dtype=int64))
(array([0, 1], dtype=int64), array([2559, 2559], dtype=int64)) (array([0, 1], dtype=int64), array([1066, 1066], dtype=int64))
(array([0, 1], dtype=int64), array([2560, 2559], dtype=int64)) (array([0, 1], dtype=int64), array([1066, 1066], dtype=int64))
(array([0, 1], dtype=int64), array([2559, 2560], dtype=int64)) (array([0, 1], dtype=int64), array([1066, 1066], dtype=int64))


In [37]:
stkFold_result

fold_ind       auc       acc       tpr       tnr        f1      ppv  \
0         0  0.495366  0.497396  0.588175  0.402556  0.544597  0.50703   
0         1  0.495366  0.497396  0.588175  0.402556  0.544597  0.50703   
0         2  0.495366  0.497396  0.588175  0.402556  0.544597  0.50703   
0         3  0.495366  0.497396  0.588175  0.402556  0.544597  0.50703   
0         4  0.495366  0.497396  0.588175  0.402556  0.544597  0.50703   

        npv      bacc       fpr       fnr       plr       nlr  
0  0.483376  0.495366  0.411825  0.597444  1.428218  0.673797  
0  0.483376  0.495366  0.411825  0.597444  1.428218  0.673797  
0  0.483376  0.495366  0.411825  0.597444  1.428218  0.673797  
0  0.483376  0.495366  0.411825  0.597444  1.428218  0.673797  
0  0.483376  0.495366  0.411825  0.597444  1.428218  0.673797

### 반복 층화 K겹 교차 검증: ```RepeatedStratifiedKFold```

In [38]:
from sklearn.model_selection import RepeatedStratifiedKFold
K = 3
R = 2
verbose = 1
scaler = RobustScaler()
rstkFold = RepeatedStratifiedKFold(n_repeats=R, n_splits=K)
list(rstkFold.split(X, y))

[(array([   1,    2,    3, ..., 6395, 6396, 6397]),
  array([   0,   10,   15, ..., 6384, 6393, 6394])),
 (array([   0,    2,    5, ..., 6394, 6395, 6396]),
  array([   1,    3,    4, ..., 6387, 6392, 6397])),
 (array([   0,    1,    3, ..., 6393, 6394, 6397]),
  array([   2,    5,    7, ..., 6391, 6395, 6396])),
 (array([   0,    1,    2, ..., 6395, 6396, 6397]),
  array([   8,    9,   14, ..., 6385, 6386, 6391])),
 (array([   0,    2,    4, ..., 6394, 6395, 6397]),
  array([   1,    3,    5, ..., 6392, 6393, 6396])),
 (array([   1,    3,    5, ..., 6392, 6393, 6396]),
  array([   0,    2,    4, ..., 6394, 6395, 6397]))]

In [39]:
verbose=1
rstkFold_result = pd.DataFrame()
for i, (train_ind, test_ind) in enumerate(rstkFold.split(X, y)): 
    X = X if isinstance(X,np.ndarray) == True else np.array(X)
    y = y if isinstance(X,np.ndarray) == True else np.array(y)
       
    X_train, X_test = X[train_ind], X[test_ind]
    y_train, y_test = y[train_ind], y[test_ind]
        
    X_train_sc = scaler.fit_transform(X_train)
    X_test_sc = scaler.transform(X_test) 
       
    knn.fit(X_train_sc, y_train)
    
    y_prob = knn.predict_proba(X_test_sc)
    bc_eval_result = binary_clf_eval(y_test,y_prob)
    bc_eval_result.insert(0, "fold_ind", i%K)
    bc_eval_result.insert(0, "repeat_ind", i//K)
    rstkFold_result = pd.concat([rstkFold_result,bc_eval_result])

    if verbose > 0:
        if i%K == 0:
            print(f'\n*** Repeat index: { i//R +1 } ***')
        print( "Fold[{}] acc: {}, sens: {}, spec: {}". format(i%K +1, bc_eval_result.acc.item(),
                                                              bc_eval_result.tpr.item(),
                                                              bc_eval_result.tnr.item()))


*** Repeat index: 1 ***
Fold[1] acc: 0.8124706985466479, sens: 0.9034676663542643, spec: 0.7213883677298312
Fold[2] acc: 0.7974683544303798, sens: 0.8968105065666041, spec: 0.6982193064667291
Fold[3] acc: 0.8166041275797373, sens: 0.9090056285178236, spec: 0.724202626641651

*** Repeat index: 2 ***
Fold[1] acc: 0.8115330520393812, sens: 0.9044048734770385, spec: 0.7185741088180112
Fold[2] acc: 0.8157524613220816, sens: 0.9080675422138836, spec: 0.7235238987816307
Fold[3] acc: 0.8161350844277674, sens: 0.9212007504690432, spec: 0.7110694183864915


In [40]:
rstkFold_result
rstkFold_result.drop('fold_ind', axis = 1).groupby(['repeat_ind']).agg([np.mean, np.std])

auc                 acc                 tpr            \
                mean       std      mean       std      mean       std   
repeat_ind                                                               
0           0.873494  0.007876  0.808848  0.010069  0.903095  0.006106   
1           0.873449  0.002476  0.814474  0.002554  0.911224  0.008832   

                 tnr                  f1            ...      bacc            \
                mean       std      mean       std  ...      mean       std   
repeat_ind                                          ...                       
0           0.714603  0.014259  0.825332  0.008571  ...  0.808849  0.010035   
1           0.717722  0.006271  0.830830  0.003023  ...  0.814473  0.002590   

                 fpr                 fnr                  plr            \
                mean       std      mean       std       mean       std   
repeat_ind                                                                
0           0.096905  0.006106  0.285397  0.014259   9.346608  0.649483   
1           0.088776  0.008832  0.282278  0.006271  10.342937  1.185462   

                 nlr            
                mean       std  
repeat_ind                      
0           2.509580  0.170653  
1           2.543774  0.078393  

[2 rows x 24 columns]

#### +) CV 결과를 코드 한 줄로 확인할 수 있는 방법: ```cross_val_score```

In [41]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle=True, test_size = 0.3)
cross_val_score(knn,
               X = X_train, y=y_train, cv=5, scoring='accuracy',verbose = 2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.3s finished


array([0.79910714, 0.80133929, 0.77790179, 0.79888268, 0.79441341])

In [47]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
scaler = RobustScaler()

pipeline = Pipeline(steps=[('scaler', scaler), 
                ('estimator', knn)])

In [48]:
pipeline

Pipeline(steps=[('scaler', RobustScaler()),
                ('estimator', KNeighborsClassifier(n_neighbors=15, p=3))])

In [49]:
cross_val_score(pipeline,
               X = X_train, y=y_train, cv=5, scoring='accuracy',verbose = 2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.5s finished


array([0.80915179, 0.80580357, 0.79799107, 0.80893855, 0.80446927])